This is using Scikit-learn's GridSearchCV on LightGBM.

This is only meant to be an early rough experiment. See the caveats in README. Using a larger learning rate or a small dataset for better speed performance.

In [2]:
from tqdm import tnrange, tqdm, tqdm_notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.ion()
import xgboost as xgb
import warnings; warnings.simplefilter('ignore')
import inspect
from sklearn.metrics import roc_auc_score, make_scorer
roc = make_scorer(roc_auc_score)
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin

In [ ]:
class MyEstimator(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators = 100, learning_rate = 0.1, max_depth=8, min_child_weight=45, reg_alpha=0.1, reg_lambda=0.1, random_state=2018, subsample =1, colsample_bytree = 1, num_leaves =44, min_split_gain =0.1
#              , eval_metric ='auc'
                ):

        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")
        for arg, val in values.items():
            setattr(self, arg, val)   
        self.fitted_ = False
    
    def fit(self, X, y=None, sample_weight=None, eval_x = None, eval_y = None):
            self.params = {'n_estimators' : self.n_estimators, 'max_depth': self.max_depth, 'learning_rate': self.learning_rate, 'min_child_weight':self.min_child_weight, 'reg_alpha':self.reg_alpha, 'reg_lambda' : self.reg_lambda, 'random_state' : self.random_state, 'subsample': self.subsample, 'colsample_bytree': self.colsample_bytree, 'num_leaves':self.num_leaves, 'min_split_gain' : self.min_split_gain
#           ,'eval_metric':self.eval_metric
            }
        self.clf = LGBMClassifier(**self.params)
        clf.fit(X, y, sample_weight,
#               eval_set = [(eval_x, eval_y)], eval_metric = 'auc', verbose = verbose, early_stopping_rounds = 200)
                )
        self.fitted_ = True
        return self

    def predict(self, X, y=None):   
        if not self.fitted_:
            raise RuntimeError("You must train classifer before predicting data!")
        pred = self.clf.predict_proba(X)[:, 1]
        return pred 

In [ ]:
train = pd.read_csv('train.csv', index_col = 0)
cols = list(train.columns)
cols.remove('TARGET')
tar = 'TARGET'

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)

gsparam = {'n_estimators' : [150], 'learning_rate' : [0.1], 'random_state' :[2018], 'num_leaves' : [44],'max_depth' :[8], 'min_child_weight' : [45],  'reg_alpha':[0.01], 'reg_lambda':[0.01], 'subsample':[1], 'colsample_bytree':[1], 'min_split_gain' : [0.1]}
           
wrapper = MyEstimator()
gs = GridSearchCV(estimator = wrapper, param_grid = gsparam, scoring = roc, cv = skf, return_train_score = False, verbose = 2, refit= False)
gs.fit(train[cols], train[tar])
pd.DataFrame(gs.cv_results_).to_csv('gridcv.csv')